# SESSION 5

Prerequisites: In a terminal, You need to create, install biopython and activate the `Conda` env as follow before to start jupyter

**We will create a new env called curso_5**

In [ ]:
!conda create -y --name curso_5

In [ ]:
!conda install -y -n curso_4 -c bioconda jupyter quast busco

In [ ]:
!conda activate curso_5

In [ ]:
!jupyter notebook &

# Quality assessment with QUAST

So far we have used `dnadiff` from the `mummer` package to evaluate our assembly given the reference genome. We obitaned the average accuracy, number of break points, number of SNPs and indels. A similar tool for meassuring the quality of an assembly is `quast`, which can be run with or without the reference file. It will yield a summary that describes how fragmented is the assembly. This is incorporated in the NG50 value (N50 without a reference), which is almost always used to compare different assemblers. It equalst the lenght of the smallest contig which together with all longer contigs covers $50\%$ of the genome. When given the reference genome, `quast` will also tell us the number of translocations, relocations and inversions, and the percentage of mismatches and indels.  
(see https://github.com/ablab/quast)

To be able to run `quast` you need a `conda` environment with `quast` package installed.

In [ ]:
!quast -h

**Replace the file bs_assembly_miniasm.fasta by the assembly you did**

In [ ]:
!quast \
    -t 4 \
    --fast \
    --silent \
    -o bs_assembly_miniasm \
    bs_assembly_miniasm.fasta

In [ ]:
!cat bs_assembly_miniasm/report.txt

In [ ]:
!quast \
    -t 4 \
    --silent \
    -o bs_quast \
    --min-identity 80.0 \
    -r data/bacillus_subtilis/bs_ref.fasta \
    bs_assembly_miniasm.fasta \
    bs_assembly_miniasm_r4.fasta \
    consensus.fasta > err 2>&1

!cat bs_quast/report.txt

In [ ]:
from IPython.display import HTML
HTML(filename="bs_quast/icarus.html")

For a more detailed summary, you need to open the `report.pdf` file from `quast` output directory.

When we do not have a reference genome and are doing *de novo* assembly, we would also like to somehow evaluate our results. For chromosome coverage and fragmentation, we can use `quast` without the reference genome to see the value of the N50 meassure. For accuracy, we can try and translate our DNA to proteins and search a protein database for matches. If our assembly is full of insertions and deletions, the open reading frames could be shifted and the resulting proteins will be without matches in the database. Luckily, there are two tools that are doing exactly this.

Now let's use the file Assemblies.zip. Unzip Assemblies, and run Quast

In [ ]:
!unzip Assemblies.zip

In [ ]:
!quast \
    -t 4 \
    --silent \
    -o bs_quast \
    --min-identity 80.0 \
    -r data/bacillus_subtilis/bs_ref.fasta \
    data/Assemblies/*

!cat bs_quast/report.txt

# Quality assessment with BUSCO

`Busco` assesses the assembly based on evolutionarily informed expectations of gene content. It searches for single copy orthologs in the determined lineage of the genome we sequenced and calculates the fractions of complete, fragmented and missing orthologs.

On the other hand, `ideel` (indels are not ideal) is a pipeline which tranlates all proteins, searches for their the best match in a protein database and calculates the length ratio between each pair. Afterwards, it draws the histogram of length ratios which should have a peak at value $1$. Assemblies that have lots of errors will have many proteins that are truncated.

Let us first see how `busco` evaluates our `racon` polished assembly and the `medaka` polished assembly. You will need a `conda` environment with `busco` packaged installed.

(see https://gitlab.com/ezlab/busco ).

In [ ]:
!busco --help

In [ ]:
!busco --list-datasets

Replace bs_assembly_miniasm.fasta by your assembly before et after correction

In [ ]:
!busco \
    -c 4 \
    -f \
    --quiet \
    -m genome \
    -l bacillales_odb10 \
    -o bs_miniasm_busco \
    -i bs_assembly_miniasm.fasta

In [ ]:
!busco \
    -c 4 \
    -f \
    --quiet \
    -m genome \
    -l bacillales_odb10 \
    -o bs_miniasm_r4_busco \
    -i bs_assembly_miniasm_r4.fasta

In [ ]:
!busco \
    -c 4 \
    -f \
    --quiet \
    -m genome \
    -l bacillales_odb10 \
    -o bs_consensus_busco \
    -i consensus.fasta

In [ ]:
Put all files called "short summary" in a same directory (my_summaries)

In [ ]:
!generate_plot.py -wd my_summaries

In [2]:
from IPython.display import Image
Image("my_summaries/busco_figure.png")

FileNotFoundError: No such file or directory: 'my_summaries/busco_figure.png'

FileNotFoundError: No such file or directory: 'my_summaries/busco_figure.png'

<IPython.core.display.Image object>

# Others tools

**Ideel**

`ideel` looks for indels. First we need to download a protein database. Usually it is the TrEMBL database, but for our purposes we will be using the much smaller one called Swiss-Prot. Run the below cell to set everything up before we can run `ideel`.  
You will need a `conda` environment with the following packages installed: `snakemake`, `diamond`, `prodigal` and `r`.

(see https://github.com/phiweger/ideel)

**YAK**

If we by any chance have short accurate reads of the same genome, we can use Yet another k-mer analyzer. Without the need of a reference genome, it creates a 31-mer histogram from the accurate data and compares it with the assembly k-mers to calculate the base accuracy (in form of Phred score). 
(see https://github.com/lh3/yak)